In [4]:
import iris
import pandas as pd


host = "localhost"
# this is the superserver port
port = 32782
namespace = "FHIRSERVER"
user = "_SYSTEM"
password = "SYS"

conn = iris.connect(
    hostname=host,
    port=port,
    namespace=namespace,
    username=user,
    password=password
)

# create a cursor
cursor = conn.cursor()

sql = """
select prac._id, prac.Key, GetFHIRPathOne(resource.ResourceString, 'Practitioner.identifier[0].value','Practitioner') As GMP from HSFHIR_X0001_S.Practitioner prac                                           join HSFHIR_X0001_R.Rsrc resource on  resource.ID = prac._id
      where IsNull(prac.addressCountry,'') <> 'US'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
pd.set_option('future.no_silent_downcasting', True)

df

,_id,Key,GMP
0,759294,Practitioner/759294,G3607596
1,759295,Practitioner/759295,G3607606
2,759296,Practitioner/759296,G3607613
3,759297,Practitioner/759297,G3607620
4,759298,Practitioner/759298,G3607637
...,...,...,...
6429,765704,Practitioner/765704,G6322764
6430,765705,Practitioner/765705,G6322771
6431,765706,Practitioner/765706,G6322788
6432,765707,Practitioner/765707,G6322795


In [5]:
import requests

headers = {"Content-Type": "application/fhir+json"}
fhirurl = "http://localhost:32783/fhir/r4"

for gpId in range(0, len(df)):
    id = df.loc[gpId,'_id']
    r = requests.delete(fhirurl+'/Practitioner/'+id, headers=headers)
